So I decided to use two diffrent API for retriving markert data and for trading in orders. yfinance allows us to access a ton of stock metrics, over 200 infact, in a very simple way. Of corse the down side being that we can't subit orders or sell. For that we are going to use the alpaca-trade-api. Alpaca has a free tier version that allows us to sell and purchases stocks, but their stock mertrics are absoultuly horendus. You can only view 5 diffrent metrics, and you can't request any information about stocks
within 15 minutes. Basicly your only allowed to viwe historical data witj alpaca. So these two api complement each other well and fill each api gaps. Great!

# yfinance is not an official API
yfinance mainly makes API calls to Yahoo Finance to gather it’s data, but it does occasionally employ HTML scraping and pandas tables scraping to unofficially gather the information off the Yahoo Finance website for some of it’s methods. As such, the functionality of some of it’s methods is at the mercy of Yahoo not changing the layout or design of some of their pages. In fact, yfinance is widely known to already have a few issues.

In [1]:
import numpy as np
import pandas as pd
import requests 
import xlsxwriter 
import math 
import alpaca_trade_api as alpaca
import datetime as dt
import pytz
import yfinance as yf
import threading
import json
import logging
from alpaca_trade_api import TimeFrame
from datetime import date
from time import sleep 

# Retrieving the S&P 500
I could use one of the avalible trading API to get a list of the index, but all the ones that offer that feature cost money. We could just import a CSV file of the list, but the index does occasionaly get updated. So I am just going to scrape wikipiedias page for the list of the current 500 companies, using pandas.

In [2]:
Wiki_List_URL = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
table = pd.read_html(Wiki_List_URL )
stocks = table[0]['Symbol'] #input only the first table, and from the table we only want the ticker Symbol of each stock
stocks

0       MMM
1       AOS
2       ABT
3      ABBV
4      ABMD
       ... 
498     YUM
499    ZBRA
500     ZBH
501    ZION
502     ZTS
Name: Symbol, Length: 503, dtype: object

In [3]:
"""
This function takes any stock symbol or ticker and retrives metrics about it such as price, volume, marketCap, and other things.
Like I mentioned in the introduction yfinance occasuionaly employs scraping. This can lead to inaccurate or wrong data.
An example would berkshire hathaway stock. Althoguh this a public comapny with shares traded on NYSE exchange, yfinance 
will return none for all of its metrics.
"""
def get_stock_info(stock_ticker):
#     if clock.is_open == False: # if the market is not open then their is not current price
#         return "Stock Market is not open"
    data = yf.Ticker(stock_ticker)
    if data.info['marketCap'] == None: # if a comapny has none marketcap. it means yfiance could not find information about it
        return 
    else:
        stock_info = pd.DataFrame({'Price':data.info['currentPrice'], 
                                   'Market Capitalization':data.info['marketCap'], 
                                   'How much to buy':'N/A'},index = [stock_ticker])
    return stock_info

In [4]:
"""
creates a table of all stock tickers and their corrosponding data in one table
"""
def table_of_stock_info(stock_list):
#     if clock.is_open == False: # if the market is not open then their is not current price
#         return "Stock Market is not open"
    table_of_stock_info = pd.DataFrame() #initalizing our variable as pandas dataframe for later
    for ticker in stock_list:
        stock_info = get_stock_info(ticker) # we get the info from yfiniace 
        table_of_stock_info = pd.concat([stock_info,table_of_stock_info]) # merge info of current 
                                                                          #stock with dataframe of all other stocks
    return table_of_stock_info
    

    


In [5]:
stock_info = table_of_stock_info(stocks[:2]) # We create a dataframe of data of all the stocks where intrested in
stock_info.to_csv('stock_metrics', index = True) # we save it to csv file

# what's next?
We basicly got a list of all s&p500 companies from wikipidea. next we used yfinace to gather information on each of these companies. We sorted our information and kept what we wanted, and then we created a list of all the comanies we gatherd info about and their metrics in one clean and orgaized pandas dateframe.

Now we need to calcualte how much stock will be buying for each comapny or our postion size. We then can save this to a file or use the alpaca api to purchases them.

In [6]:
portfolio_size = input("Enter the value of your portfolio:") # we ask the user for their inteded portfolio size
while True:
    try:
        portfolio_size = float(portfolio_size)
        break
    except ValueError: # if the user types anything but an integer we tell them do type an integer
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")
    

Enter the value of your portfolio:
That's not a number! 
 Try again:
Enter the value of your portfolio:100000


# calculate postion size and indivuial stocks purchases

In [7]:
postion_size = portfolio_size / (len(stock_info)) # The amount of money we will be spending on each indiviudal comapny
for i in stock_info.index: 
     stock_info.loc[i, 'How much to buy'] = math.floor(postion_size / stock_info['Price'][i]) # we calculate how much stock to 
                                                                                              # buy for each comapny
    

# Set up Alpaca and begin trading
After your done setting up your account and creating your api keys.Just fill in the details below. Just a heads up. I've hardcoded my API key and secret since I was testing Alpaca out with a paper trading account, and I have deactivided these since the publishing of this project on github. In a live environment, however, it is a good idea to take the extra security precaution of storing your authentication details in environment variables. This a good article that explain why and how.[How to set environment variables](https://www.twilio.com/blog/2017/01/how-to-set-environment-variables.html)

In [8]:
logging.basicConfig(
    filename='errlog.log',
    level=logging.WARNING,
    format='%(asctime)s:%(levelname)s:%(message)s',
)
# authentication and connection details
api_key ='fsidfsindfisudfbdsiuf'   #Insert_your_api_key_here
api_secret ='oasdfnsadifn;isadfnasidfnslaidnf;sadnf'  #Insert_your_api_secret_here
base_url ='https://paper-api.alpaca.markets'    #https://paper-api.alpaca.markets

# instantiate REST API
api = alpaca.REST(api_key, api_secret, base_url, api_version='v2')

# obtain account information
account = api.get_account()

#initiate varibles used later on
trade_msg = []
order_msg = []
past_trades = []

searching_for_trade = False
order_sent = False
order_submitted = False
active_trade = False
done_for_the_day = False

In [ ]:
# we want to cancel all orders in case our script got interrupted for 
#whatever reason and we have to restart in the middle of a market day.
# check if market is open
# check if market is open
api.cancel_all_orders()
clock = api.get_clock()

if clock.is_open:
    pass
else:
    time_to_open = clock.next_open - clock.timestamp
    sleep(time_to_open.total_seconds())

if len(api.list_positions()) == 0:
    searching_for_trade = True
else:
    active_trade = True

# init WebSocket
conn = tradeapi.stream2.StreamConn(api_key, api_secret, base_url)


@conn.on(r'^account_updates$')
async def on_account_updates(conn, channel, account):
    order_msg.append(account)


@conn.on(r'^trade_updates$')
async def on_trade_updates(conn, channel, trade):
    trade_msg.append(trade)
    if 'fill' in trade.event:
        past_trades.append(
            [
                trade.order['updated_at'],
                trade.order['symbol'],
                trade.order['side'],
                trade.order['filled_qty'],
                trade.order['filled_avg_price'],
            ]
        )
        with open('past_trades.csv', 'w') as f:
            json.dump(past_trades, f, indent=4)
        print(past_trades[-1])


def ws_start():
    conn.run(['account_updates', 'trade_updates'])


# start WebSocket in a thread
ws_thread = threading.Thread(target=ws_start, daemon=True)
ws_thread.start()
sleep(10)


# functions
def time_to_market_close():
    clock = api.get_clock()
    closing = clock.next_close - clock.timestamp
    return round(closing.total_seconds() / 60)


def send_order(direction):
    if time_to_market_close() > 20:
        if direction == 'buy':
            sl = high - range_size
            tp = high + range_size
        elif direction == 'sell':
            sl = low + range_size
            tp = low - range_size

        api.submit_order(
            symbol='AAPL',
            qty=100,
            side=direction,
            type='market',
            time_in_force='day',
            order_class='bracket',
            stop_loss=dict(stop_price=str(sl)),
            take_profit=dict(limit_price=str(tp)),
        )
        return True, False

    else:
        return False, True


# main loop
while True:

    try:

        candlesticks = api.get_barset('AAPL', 'minute', limit=10)
        high = candlesticks['AAPL'][0].h
        low = candlesticks['AAPL'][0].l
        range_size = high - low
        if range_size / candlesticks['AAPL'][0].c < 0.003:
            range_size = candlesticks['AAPL'][0].c * 0.003
        for candle in candlesticks['AAPL']:
            if candle.h > high:
                high = candle.h
            elif candle.l < low:
                low = candle.l
        range_size = high - low

        while searching_for_trade:
            clock = api.get_clock()
            sleep(60 - clock.timestamp.second)
            candlesticks = api.get_barset('AAPL', 'minute', limit=1)
            if candlesticks['AAPL'][0].c > high:
                searching_for_trade = False
                order_sent, done_for_the_day = send_order('buy')

            elif candlesticks['AAPL'][0].c < low:
                searching_for_trade = False
                order_sent, done_for_the_day = send_order('sell')

        while order_sent:
            sleep(1)
            for item in trade_msg:
                if item.event == 'new':
                    order_submitted = True
                    order_sent = False

        while order_submitted:
            sleep(1)
            for item in trade_msg:
                if item.order['filled_qty'] == '100':
                    order_submitted = False
                    active_trade = True
                    trade_msg = []

        while active_trade:
            for i in range(time_to_market_close() - 5):
                sleep(60)
                if len(api.list_positions()) == 0:
                    active_trade = False
                    searching_for_trade = True
                    break
            if active_trade:
                done_for_the_day = True
                active_trade = False

        while done_for_the_day:
            api.close_all_positions()
            clock = api.get_clock()
            next_market_open = clock.next_open - clock.timestamp
            sleep(next_market_open.total_seconds())
            searching_for_trade = True

    except Exception as e:
        logging.exception(e)